In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
engine = create_engine('sqlite:///hawaii.sqlite')

In [ ]:
# Automap Base creates a base class for an automap schema in SQLAlchemy (reflect an existing database into a new model)
Base = automap_base()

# reflect our tables with the prepare() function.
Base.prepare(engine, reflect=True)

In [ ]:
# We can view all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station  = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
results = []
results = session.query(Measurement.date, Measurement.prcp)
print(f'most recent date: {max(results.all())}')
print(results.all())

In [ ]:
# Starting from the last data point in the database. 
# Calculate the date one year from the last date in data set.
results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= prev_year)
print(f'\n{results}')

In [ ]:
# SELECVT * FROM 
temp = session.query(Measurement)
print(temp)


In [ ]:
# Perform a query to retrieve the data and precipitation scores
results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= prev_year).all()
print(f'\n{results}')

In [ ]:
# Save the query results as a Pandas DataFrame 
df = pd.DataFrame(results, columns=['date','precipitation'])
df.head()

In [ ]:
# set the index to the date column
df.set_index(df['date'], inplace=True)
df.head()

In [ ]:
# print DF without index
print(df.to_string(index=False))

In [ ]:
# Sort the dataframe by date
df = df.sort_index()
df.head()

In [ ]:
print(df.to_string(index=False))

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
df.plot(rot=90)

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
df.describe()

In [ ]:
# How many stations are available in this dataset?
session.query(func.count(Station.station)).all()

In [ ]:

# What are the most active stations?
session.query(Measurement.station, func.count(Measurement.station))

In [ ]:
# List the stations and the counts in descending order.
session.query(Measurement.station, func.count(Measurement.station)).\
group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature most active station?
session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
filter(Measurement.station == 'USC00519281').all()

In [ ]:
# Choose the station with the highest number of temperature observations.
results = session.query(Measurement.tobs).filter(Measurement.station == 'USC00519281')
# Query the last 12 months of temperature observation data
results = session.query(Measurement.tobs).filter(Measurement.station == 'USC00519281').\
filter(Measurement.date >= prev_year).all()
print(results)

In [ ]:

#  for this station and plot the results as a histogram
df = pd.DataFrame(results, columns=['tobs'])
print(df)
df.plot.hist(bins=12)
plt.tight_layout()

In [ ]:
# num of bin to 5
df.plot.hist(bins=5)
plt.tight_layout()

In [ ]:
# num of bins to 20
df.plot.hist(bins=20)
plt.tight_layout()

In [ ]:
# Write a function called `calc_temps` that will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start, end):
   
    # checking for correct date format
    try:
        dt.datetime.strptime(start,'%Y-%m-%d')
    except ValueError:
        # raise ValueError("Incorrect start data format, should be YYYY-MM-DD")
        return "Incorrect start data format, should be YYYY-MM-DD"
        
    try:
        dt.datetime.strptime(end,'%Y-%m-%d')
    except ValueError:
        # raise ValueError("Incorrect end data format, should be YYYY-MM-DD")
        return "Incorrect end data format, should be YYYY-MM-DD"
    
    
    results = session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
    filter(Measurement.date >= start).filter(Measurement.date <= end).all()
    return results

In [ ]:

start = '2017-06-01'
end = '2017-06-30'
print(calc_temps(start, end))

In [ ]:
# function returns stat for specified start and end date.
def key_stat_data(start, end):
    '''
    This function get temperature information from measurement between specified start and end date.
    Returns data frame
    '''
    try:
        results = session.query(Measurement.tobs, Measurement.prcp).filter(Measurement.date >= start).filter(Measurement.date <= end).all()
        stat_data_df = pd.DataFrame(results, columns=['tobs', 'prcp'])
        return stat_data_df
    except:
        print('Error in getting temperature information from measurement')
        return None

In [ ]:
def get_min_max_years():
    '''
    This function gets start and end year from Measurement
    '''
    try:
        results = session.query(func.min(Measurement.date), func.max(Measurement.date)).all()
        return results[0][0].split('-')[0], results[0][1].split('-')[0]
    except:
        print('Error in getting min or max date from measurement')
        return None

In [ ]:
get_min_max_years()

In [ ]:

def get_stat_for_month(mon_val):
    '''
    This will get stat for a specified month for all years and return dataframe with information
    '''
    if len(mon_val) != 2:
        print('Error in month format')
        return None
    try:
        year_range = get_min_max_years()
        start = f'{year_range[0]}-{mon_val}-01'
        end = f'{year_range[1]}-{mon_val}-30'
        return key_stat_data(start, end)
    except:
        print('Error in getting temperature information from measurement')
        return None


In [ ]:
# getting tobs for all years and for all stations in month of june
june_df = get_stat_for_month('06')
june_df

In [ ]:
# month of june information
june_df.describe()

In [ ]:
june_df['tobs'].describe()

In [ ]:
june_df.columns

In [ ]:
june_df['tobs'].plot.hist()
# june_df.plot.hist()
plt.xlabel('Temperature')
plt.title('Temperature in Month of June')
plt.tight_layout()

In [ ]:
dec_df = get_stat_for_month('12')
dec_df

In [ ]:
# month of dec information
dec_df.describe()

In [ ]:
dec_df['tobs'].describe()

In [ ]:
dec_df['tobs'].plot.hist()
plt.xlabel('Temperature')
plt.title('Temperature in Month of December')
plt.tight_layout()

In [ ]:
june_df['prcp'].plot.hist()
# june_df.plot.hist()
plt.xlabel('Precipitation')
plt.title('Precipitation in Month of June')
plt.tight_layout()

In [ ]:
dec_df['prcp'].plot.hist()
plt.xlabel('Precipitation')
plt.title('Precipitation in Month of December')
plt.tight_layout()